Programmer: Giovanni Vecchione
Date: 2/12/24
Subject: Project 1

Create a Deep Neural Networks model using the data set of your choice. Model should have at least 4 hidden layers. Dropout and Batch Normalization layers are not counted.

In [1]:
import tensorflow as tf
import matplotlib as mpl

#Step 1 - Prep the data

#import data
fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()

#split the data between training and test sets, and leave some left over for training set validation
#training set (60,000 images) and a test set (10,000 images), but well hold out the last 5,000 images from the training set for validation

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

#scale and label if needed

#Step 2 - Create the neural net structure

tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28, 28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

model.summary()

#if you want to see layers or weights use get_layer() and get_weights()

#Step 3 - Compile the model

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

#Step 4 - Train and Evaluate model
#use fit() to train, epochs test how many trials occur
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

#Step 5 - Check for overfitting and retest if needed (STOP HERE BASED ON ASSIGNEMENT REQUIREMENTS)

#Step 6 - Visualize the data

#Step 7 - Use the model to make predictions